# Training & Testing Dataset Preparation

### Metadata

In [2]:
import os
import numpy

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# read in train data
TRAINING_DATA_PATH = '../2025_A2/train'
TRAIN_LABEL_PATH = TRAINING_DATA_PATH + '/train_metadata.csv'
IMAGE_DIMENTION = (64, 64)

### Function Definitions



In [3]:
def get_labels(labels_path):
    # load labels as dict 
    labels = {}
    mxm_labels = 0
    with open(labels_path, 'r') as f:
        counter:int = 0
        for line in f:
            # skip the first line
            if counter == 0:
                counter += 1
                continue
            line = line.strip().split(',')
            # line [1] is the image name, line [2] is the label
            labels[line[1]] = int(line[2])
            if int(line[2]) > mxm_labels:
                mxm_labels = int(line[2])
    print(f"Max label: {mxm_labels}")
    return labels

def load_and_preprocess_images(image_folder, labels_dict, target_size=IMAGE_DIMENTION):
    images = []
    labels = []
    
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg"): 
            img_path = os.path.join(image_folder, filename)
            
            # Read image with TensorFlow
            img = tf.io.read_file(img_path)  # Read the image file
            img = tf.image.decode_jpeg(img, channels=3)  # Decode the JPEG image (for JPG files)

            # Resize the image to the target size (100x100)
            img_resized = tf.image.resize(img, target_size)
            
            # Normalize image to [0, 1]
            img_normalized = img_resized / 255.0
            images.append(img_normalized)
            
            # Get the label from the dictionary
            label = labels_dict.get(filename)
            labels.append(label)
    
    # Convert lists to numpy arrays
    images = numpy.array(images)
    labels = numpy.array(labels)
    
    return images, labels

### Prepare Data

In [4]:
overall_images, overall_labels = load_and_preprocess_images(TRAINING_DATA_PATH, get_labels(TRAIN_LABEL_PATH))

train_images, test_images = tf.keras.utils.split_dataset(
    overall_images, left_size=0.9)

train_labels, test_labels = tf.keras.utils.split_dataset(
    overall_labels, left_size=0.9)


Max label: 42


# Model Definition & Model Training

### Definition

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (9, 9), activation='relu', 
                        input_shape=(IMAGE_DIMENTION[0], IMAGE_DIMENTION[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (6, 6), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(43, activation='softmax'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 56, 56, 32)        7808      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 28, 28, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 64)        73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 11, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 64)         0

### Train

In [6]:
# 1. Log everything to TensorBoard
tb = tf.keras.callbacks.TensorBoard(log_dir="logs", histogram_freq=1, write_images=True)

# tarin the model
model.fit(numpy.array(list(train_images)),
          numpy.array(list(train_labels)),
          epochs=10, batch_size=64, callbacks=[tb],
          validation_data=(numpy.array(list(train_images)), numpy.array(list(train_labels))))

Epoch 1/10


C:\Users\Administrator\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


78/78 [==============================] - 10s 125ms/step - loss: 3.2980 - accuracy: 0.1597 - val_loss: 2.5748 - val_accuracy: 0.3817
Epoch 2/10
78/78 [==============================] - 10s 122ms/step - loss: 2.0191 - accuracy: 0.4902 - val_loss: 1.3761 - val_accuracy: 0.6404
Epoch 3/10
78/78 [==============================] - 10s 122ms/step - loss: 1.1004 - accuracy: 0.7070 - val_loss: 0.7479 - val_accuracy: 0.7961
Epoch 4/10
78/78 [==============================] - 10s 122ms/step - loss: 0.6906 - accuracy: 0.8107 - val_loss: 0.4703 - val_accuracy: 0.8702
Epoch 5/10
78/78 [==============================] - 10s 124ms/step - loss: 0.4486 - accuracy: 0.8775 - val_loss: 0.4301 - val_accuracy: 0.8799
Epoch 6/10
78/78 [==============================] - 10s 125ms/step - loss: 0.3321 - accuracy: 0.9105 - val_loss: 0.2646 - val_accuracy: 0.9287
Epoch 7/10
78/78 [==============================] - 10s 126ms/step - loss: 0.2242 - accuracy: 0.9382 - val_loss: 0.2149 - val_accuracy: 0.9348
Epoch 8/10

In [7]:
model.save('../models/cnn_model.h5')

# Evaluation

In [8]:
validation_loss, validation_accuracy = model.evaluate(numpy.array(list(test_images)), numpy.array(list(test_labels)), batch_size=32)

print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

18/18 [==============================] - 0s 19ms/step - loss: 0.4528 - accuracy: 0.8761
Validation Loss: 0.45281821489334106
Validation Accuracy: 0.87613844871521
